<h1>Import Necessary Packages</h1>

In [ ]:
import sys
sys.path.append("../")
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import json
import os
import ast
from app.configs import *
from scripts.fmd import FMDCalculator
from scripts.utils import render_custom_radiomap, render_colorbar
from scripts.argos_optimize import ArgosOptimization
import pandas as pd
from sionna.rt import load_scene, PlanarArray, Receiver, Transmitter, PathSolver, RadioMap
import mitsuba as mi
import drjit as dr

<h1>Load Scene and Assign Material Properties</h1>

In [ ]:
#Load scene
scene=None
try:
    files = os.listdir(cfg_mesh_directory)
except FileNotFoundError:
    raise Exception('Directory not found! Check configs.py and put proper cfg_mesh_directory')
xml_file = None
jsonfile = None
for f in files:
    if f.endswith('.xml'):
        xml_file = f
        print(f"XML file found: {cfg_mesh_directory}/{f}")
        scene = load_scene(f"{cfg_mesh_directory}/{f}")
        print("Scene loaded!")
    if f.endswith('.json'):
        jsonfile = f
        print(f"Material Properties JSON file found: {cfg_mesh_directory}/{f}")
if xml_file is None:
    raise Exception('No xml file found! Check configs.py and put proper cfg_mesh_directory')

#Assign Material Property
with open(f"{cfg_mesh_directory}/{jsonfile}", "r", encoding="utf-8") as f:
    mat_props = json.load(f)
for obj_name in list(mat_props.keys()):
    obj = scene.get(obj_name)
    obj.radio_material = mat_props[obj_name]
    obj.radio_material.color = cfg_color_dict[mat_props[obj_name]]

#Remove Bounding Box and Floor
bounding_box = scene.get(cfg_bounding_box_name)
if cfg_set_bounding_box_transparent:
    scene.edit(remove=bounding_box)
    
floor = scene.get(cfg_floor_name)
if cfg_set_floor_transparent:
    scene.edit(remove=floor)

#Import FMD Cache
fmd = FMDCalculator(
    arena_size=(cfg_scene_length, cfg_scene_breadth), spacing=0.5,
    real_csv_path=cfg_cir_path,
    scene_path=f"{cfg_mesh_directory}/{xml_file}"
)
fmd.import_cache_from_json(cfg_fmd_cache_name)
tx_list = [ast.literal_eval(k) for k in fmd._cache.keys()]
prior_map = np.load(cfg_prior_map)

<h1>View the Prior Map</h1>

In [ ]:
rm = render_custom_radiomap(prior_map.copy(), scene, center=[cfg_scene_length/2, 0.1, cfg_scene_breadth/2], size=[cfg_scene_length, cfg_scene_breadth],invert=True)
render_colorbar(prior_map.copy(),"Priors")
scene.preview(resolution=(1920,1080),radio_map=rm,clip_at=cfg_clip_at, clip_plane_orientation=cfg_clip_plane_orientation,rm_metric="rss")

<h1>Run Optimization</h1>

In [ ]:
opt = ArgosOptimization(
    fmd_calc=fmd,
    candidate_txs=tx_list,
    prior_map=prior_map,
    rssi_threshold=-92.0,  # RSSI cutoff
    min_anchors=3,         # need at least 3 anchors for trilateration
)
res = opt.run(budget=5)

<h1>See the FMD Map after anchor selection. Brighter = lower FMD</h1>

In [ ]:
tx_list_ = res['selected']
error_map = res['fmd_map'].copy()
for i in list(scene.transmitters.keys()):
    scene.remove(i)
for i in range(len(tx_list_)):    
    tx = Transmitter(name=f"tx_{i}", position=mi.Point3f(float(tx_list_[i][0]),float(tx_list_[i][1]),float(tx_list_[i][2])), display_radius=500, look_at=[0,0,0])
    scene.add(tx)

rm = render_custom_radiomap(error_map, scene, center=[cfg_scene_length/2, 0.1, cfg_scene_breadth/2], size=[cfg_scene_length, cfg_scene_breadth], invert=True)
render_colorbar(res['fmd_map'],"$\\Delta_\\text{FMD} (Taps)$", invert=True)
scene.preview(resolution=(1920,1080),radio_map=rm,clip_at=cfg_clip_at, clip_plane_orientation=cfg_clip_plane_orientation,rm_metric="rss")

<h1>See the FMD Map overlayed on the prior map</h1>

In [ ]:
overlay = prior_map.copy() * res['fmd_map'].copy()
overlay_mask = (overlay==0)
overlay[overlay_mask] = np.max(overlay)+1
for i in range(len(tx_list_)):
    scene.remove(f"tx_{i}")
    tx = Transmitter(name=f"tx_{i}", position=mi.Point3f(float(tx_list_[i][0]),float(tx_list_[i][1]),float(tx_list_[i][2])), display_radius=500, look_at=[0,0,0])
    scene.add(tx)

rm = render_custom_radiomap(overlay.copy(), scene, center=[cfg_scene_length/2, 0.1, cfg_scene_breadth/2], size=[cfg_scene_length, cfg_scene_breadth],invert=True)
render_colorbar(overlay, "$\\Delta_\\text{FMD} (Taps)$", invert=True)
scene.preview(resolution=(1920,1080),radio_map=rm,clip_at=cfg_clip_at, clip_plane_orientation=cfg_clip_plane_orientation,rm_metric="rss")